In [30]:
import sys
sys.path.append('..')
from config_notebooks import *
from qdrant.qdrant_helper import *
from azureopenaimanager.azureopenai_helper import AzureOpenAIManager
from azure_blob.azure_blob_helper import AzureBlobHelper

In [31]:
search = QdrantHelper(QDRANT_URL,QDRANT_KEY,MODEL_NAME,QDRANT_COLLECTION)

In [32]:
query = """
What is Gene Therapy ?
"""

In [33]:
results_to_return, \
metadata_source_filename_to_return, \
metadata_source_page_to_return, \
reranker_score_to_return = search.get_search_results(user_input=query,CATEGORY="")


What is Gene Therapy ?

################################
[-7.56286606e-02  6.96051493e-03 -3.21705565e-02  1.62467882e-02
 -2.45563965e-02 -1.46129709e-02  1.65800173e-02  9.32865515e-02
 -7.13963411e-04  2.87732948e-02 -2.06096098e-02  9.12972838e-02
 -4.89719510e-02 -1.60598680e-02 -9.05137733e-02  1.07927993e-01
 -4.66796122e-02  5.43054640e-02  5.97447827e-02 -3.87682579e-02
 -5.62033318e-02  8.25572759e-02  7.50292093e-02  9.05618444e-03
 -3.41565423e-02  1.72764622e-02 -4.94904397e-03 -1.98379122e-02
 -4.91072796e-03  1.96633022e-02  3.43820155e-02  1.15315113e-02
 -1.99043751e-02  5.04070446e-02 -6.87898919e-02  7.65125006e-02
 -5.73197566e-02  2.34549884e-02 -2.94078365e-02 -2.16272026e-02
  2.91770454e-02 -1.66741591e-02 -2.80940756e-02  3.78320366e-02
  5.84611595e-02 -4.91465889e-02 -5.56746721e-02 -4.33879942e-02
 -9.71481088e-04  1.69822685e-02 -2.78769676e-02 -1.65191274e-02
 -1.72071159e-02 -4.21385132e-02  1.81118939e-02 -5.11367689e-05
  1.70102064e-02 -5.30016385e-02

In [34]:
metadata_source_filename_to_return

['BIOTECH.pdf', 'BIOTECH.pdf', 'BIOTECH.pdf', 'BIOTECH.pdf', 'BIOTECH.pdf']

In [35]:
metadata_source_page_to_return

['BIOTECH-5.pdf',
 'BIOTECH-9.pdf',
 'BIOTECH-9.pdf',
 'BIOTECH-5.pdf',
 'BIOTECH-6.pdf']

In [36]:
reranker_score_to_return

[]

In [37]:
results_to_return[0]

'2 Gene Therapy\nIf a person is born with a hereditary disease, can a corrective therapy\nbe taken for such a disease? Gene therapy is an attempt to do this.\nGene therapy is a collection of methods that allows correction of a\ngene defect that has been diagnosed in a child/embryo. Here genes\nare inserted into a person’s  cells and tissues to treat a disease.\nCorrection of a genetic defect involves delivery of a normal gene into\nthe individual or embryo to take over the function of and compensate\nfor the non-functional gene.\nThe first clinical gene therapy was given in 1990 to a 4-year old girl\nwith adenosine deaminase (ADA) deficiency. This enzyme is crucial for\nthe immune system to function. The disorder is caused due to the deletion\nof the gene for adenosine deaminase. In some children ADA deficiency\ncan be cured by bone marrow transplantation; in others it can be treated\nby enzyme replacement therapy, in which functional ADA is given to the\npatient by injection. But the 

In [38]:
def get_reply(user_input, content):

    azure_open_ai_manager = AzureOpenAIManager(
                    endpoint=AZURE_OPENAI_ENDPOINT,
                    api_key=AZURE_OPENAI_KEY,
                    deployment_id=AZURE_OPENAI_DEPLOYMENT_ID,
                    api_version="2023-05-15"
                )
    
    conversation=[{"role": "system", "content": "If the answer is not found within the context, please mention \
        that the answer is not found \
        Do not answer anything which is not in the context"}]
    reply = azure_open_ai_manager.generate_reply_from_context(user_input, content, conversation)
    return reply

In [39]:
azure_blob_helper = AzureBlobHelper(AZ_ST_ACC_NAME,
                                    AZ_ST_ACC_KEY,
                                    AZ_ST_CONTAINER_NAME)

In [40]:
context = "\n".join(results_to_return)
reply = get_reply(query, context)

URLs = []

reranker_confidence = ""

for page in metadata_source_page_to_return:
    URLs.append(azure_blob_helper.generate_sas_url(page))


In [41]:
reply

('Gene therapy is a collection of methods that allows correction of a gene defect that has been diagnosed in a child/embryo. It involves the insertion of genes into an individual’s cells and tissues to treat diseases, especially hereditary diseases. The goal of gene therapy is to replace a defective mutant allele with a functional one or to target specific genes for amplification. Viruses are often used as vectors to transfer healthy genes or portions of genes into the host cells. The first clinical gene therapy was given in 1990 to a 4-year-old girl with adenosine deaminase (ADA) deficiency.',
 1407,
 1285,
 122)